<a href="https://colab.research.google.com/github/KAVYANSHTYAGI/Food-Image-Classifier/blob/main/novel_architecture_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import os
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import tensorflow_hub as hub
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau , LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import numpy as np
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import accuracy_score,classification_report
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.utils import to_categorical

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:

base_dir_training = "/content/drive/MyDrive/Food Image Datasets/North Indian/main_final/training"
class_labels_training = os.listdir(base_dir_training)
print(len(class_labels_training))

base_dir_testing = "/content/drive/MyDrive/Food Image Datasets/North Indian/main_final/testing"
class_labels_testing = os.listdir(base_dir_testing)
print(len(class_labels_testing))

30
30


In [4]:
#training

data_training = []
count = 0

for label in class_labels_training:
    path = os.path.join(base_dir_training, label)
    print(path)
    for img in os.listdir(path):
        try:
            image = load_img(os.path.join(path,img), color_mode="rgb", target_size=(224,224))
            image = img_to_array(image)
            image /= 255.0
            data_training.append([image, count])

        except Exception as e:
            pass
    count = count + 1

#testing

data_testing = []
count = 0

for label in class_labels_testing:
    path = os.path.join(base_dir_testing, label)
    print(path)
    for img in os.listdir(path):
        try:
            image = load_img(os.path.join(path,img), color_mode="rgb", target_size=(224,224))
            image = img_to_array(image)
            image /= 255.0
            data_testing.append([image, count])

        except Exception as e:
            pass
    count = count + 1


/content/drive/MyDrive/Food Image Datasets/North Indian/main_final/training/Aloo Puri
/content/drive/MyDrive/Food Image Datasets/North Indian/main_final/training/bal mithai - Google Search
/content/drive/MyDrive/Food Image Datasets/North Indian/main_final/training/Bhindi Masala
/content/drive/MyDrive/Food Image Datasets/North Indian/main_final/training/Chhole Bhature
/content/drive/MyDrive/Food Image Datasets/North Indian/main_final/training/chole kulcha food images - Google Search
/content/drive/MyDrive/Food Image Datasets/North Indian/main_final/training/Dal Makhni
/content/drive/MyDrive/Food Image Datasets/North Indian/main_final/training/egg bhurji - Google Search
/content/drive/MyDrive/Food Image Datasets/North Indian/main_final/training/ghewar - Google Search
/content/drive/MyDrive/Food Image Datasets/North Indian/main_final/training/Gulab Jamun
/content/drive/MyDrive/Food Image Datasets/North Indian/main_final/training/kathi roll - Google Search
/content/drive/MyDrive/Food Image

In [5]:
print(len(data_training))
print(len(data_testing))
print(count)
num_classes = count

8039
2027
30


In [6]:
X_train,y_train = zip(*data_training)

x_train = np.array(X_train)
y_train = np.array(y_train)


X_test,y_test = zip(*data_testing)

x_test = np.array(X_test)
y_test = np.array(y_test)

In [7]:
y_test = to_categorical(y_test, num_classes=num_classes)
y_train = to_categorical(y_train, num_classes=num_classes)

In [8]:
train_datagen = ImageDataGenerator(
    rotation_range=90,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'               # Fill missing pixels after transformations
)


val_test_datagen = ImageDataGenerator()


train_generator = train_datagen.flow(x_train, y_train)
val_generator = val_test_datagen.flow(x_test, y_test)

In [44]:
import tensorflow as tf
import numpy as np

def multi_input_dataset(generator, num_inputs):
    """
    Converts an ImageDataGenerator into a tf.data.Dataset that supports multiple inputs.

    Args:
    - generator: The ImageDataGenerator that yields (images, labels).
    - num_inputs: Number of model inputs.

    Returns:
    - A tf.data.Dataset that provides multiple inputs.
    """

    def _generator():
        for batch_images, batch_labels in generator:
            # Duplicate input for all three branches
            yield [batch_images] * num_inputs, batch_labels

    # ✅ Correct way to get the number of classes
    num_classes = len(generator.class_indices)

    # Create dataset
    dataset = tf.data.Dataset.from_generator(
        _generator,
        output_signature=(
            [tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32)] * num_inputs,
            tf.TensorSpec(shape=(None, num_classes), dtype=tf.float32)  # ✅ Uses `len(generator.class_indices)`
        )
    )

    return dataset


In [62]:
import tensorflow as tf
import numpy as np

# Define the number of model inputs
NUM_INPUTS = 3

def data_generator(x_data, y_data):
    """ Generator that yields images correctly for multi-input models. """
    for img, label in zip(x_data, y_data):
        yield (img, img, img), label  # Three separate inputs, NOT an extra dimension



In [66]:
train_multi_input = None
val_multi_input = None

In [67]:
# Get input shape dynamically
input_shape = (224, 224, 3)  # Ensure correct shape
num_classes = y_train.shape[-1]

# Define dataset correctly
train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(x_train, y_train),
    output_signature=(
        (tf.TensorSpec(shape=input_shape, dtype=tf.float32),  # Input 1
         tf.TensorSpec(shape=input_shape, dtype=tf.float32),  # Input 2
         tf.TensorSpec(shape=input_shape, dtype=tf.float32)),  # Input 3
        tf.TensorSpec(shape=(num_classes,), dtype=tf.float32)  # Labels
    )
)

val_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(x_test, y_test),
    output_signature=(
        (tf.TensorSpec(shape=input_shape, dtype=tf.float32),  # Input 1
         tf.TensorSpec(shape=input_shape, dtype=tf.float32),  # Input 2
         tf.TensorSpec(shape=input_shape, dtype=tf.float32)),  # Input 3
        tf.TensorSpec(shape=(num_classes,), dtype=tf.float32)  # Labels
    )
)

# Batch and optimize performance
train_multi_input = train_dataset.batch(64).prefetch(tf.data.AUTOTUNE)
val_multi_input = val_dataset.batch(64).prefetch(tf.data.AUTOTUNE)


In [68]:
for images, labels in train_multi_input.take(1):
    print(f"Texture Input Shape: {images[0].shape}")  # Should be (batch_size, 224, 224, 3)
    print(f"Color Input Shape: {images[1].shape}")    # Should be (batch_size, 224, 224, 3)
    print(f"Structure Input Shape: {images[2].shape}") # Should be (batch_size, 224, 224, 3)


Texture Input Shape: (64, 224, 224, 3)
Color Input Shape: (64, 224, 224, 3)
Structure Input Shape: (64, 224, 224, 3)


model

In [118]:
import tensorflow as tf
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras import layers

# Define three separate inputs
texture_input = Input(shape=(224, 224, 3), name="texture_input")
color_input = Input(shape=(224, 224, 3), name="color_input")
structure_input = Input(shape=(224, 224, 3), name="structure_input")


In [14]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Conv2D, GlobalAveragePooling2D, GlobalMaxPooling2D, Dense, Multiply, Reshape, Add, Activation

class CBAMBlock(Layer):
    def __init__(self, ratio=8, **kwargs):
        super(CBAMBlock, self).__init__(**kwargs)
        self.ratio = ratio

    def build(self, input_shape):
        channel = input_shape[-1]

        # Channel Attention Layers
        self.avg_dense1 = Dense(channel // self.ratio, activation='relu', use_bias=False)
        self.avg_dense2 = Dense(channel, activation='sigmoid', use_bias=False)

        self.max_dense1 = Dense(channel // self.ratio, activation='relu', use_bias=False)
        self.max_dense2 = Dense(channel, activation='sigmoid', use_bias=False)

        # Spatial Attention Layer
        self.spatial_conv = Conv2D(1, kernel_size=7, padding="same", activation='sigmoid', use_bias=False)

    def call(self, inputs):
        # Channel Attention
        avg_pool = GlobalAveragePooling2D()(inputs)
        max_pool = GlobalMaxPooling2D()(inputs)

        avg_dense = self.avg_dense2(self.avg_dense1(avg_pool))
        max_dense = self.max_dense2(self.max_dense1(max_pool))

        channel_attention = Add()([avg_dense, max_dense])
        channel_attention = Activation('sigmoid')(channel_attention)
        channel_attention = Multiply()([inputs, channel_attention])

        # Spatial Attention
        avg_pool = tf.reduce_mean(channel_attention, axis=-1, keepdims=True)
        max_pool = tf.reduce_max(channel_attention, axis=-1, keepdims=True)
        concat = tf.concat([avg_pool, max_pool], axis=-1)

        spatial_attention = self.spatial_conv(concat)
        spatial_attention = Multiply()([channel_attention, spatial_attention])

        return spatial_attention


 Branch 1: Texture Branch (EfficientNetB4 + CBAM Attention)
Purpose: Local texture extraction
Model: EfficientNetB4 (CNN backbone)

In [119]:
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.layers import Conv2D, GlobalMaxPooling2D, Multiply, Add, Activation

# Load Pretrained EfficientNetB4 (Feature Extractor)
texture_base = EfficientNetB4(include_top=False, weights='imagenet')(texture_input)

# ✅ Apply CBAM Attention (Custom Block)
class CBAMBlock(layers.Layer):
    def __init__(self, ratio=8, **kwargs):
        super(CBAMBlock, self).__init__(**kwargs)
        self.ratio = ratio

    def build(self, input_shape):
        channel = input_shape[-1]

        # Channel Attention
        self.avg_dense1 = Dense(channel // self.ratio, activation='relu', use_bias=False)
        self.avg_dense2 = Dense(channel, activation='sigmoid', use_bias=False)

        self.max_dense1 = Dense(channel // self.ratio, activation='relu', use_bias=False)
        self.max_dense2 = Dense(channel, activation='sigmoid', use_bias=False)

        # Spatial Attention
        self.spatial_conv = Conv2D(1, kernel_size=7, padding="same", activation='sigmoid', use_bias=False)

    def call(self, inputs):
        # Channel Attention
        avg_pool = GlobalAveragePooling2D()(inputs)
        max_pool = GlobalMaxPooling2D()(inputs)

        avg_dense = self.avg_dense2(self.avg_dense1(avg_pool))
        max_dense = self.max_dense2(self.max_dense1(max_pool))

        channel_attention = Add()([avg_dense, max_dense])
        channel_attention = Activation('sigmoid')(channel_attention)
        channel_attention = Multiply()([inputs, channel_attention])

        # Spatial Attention
        avg_pool = tf.reduce_mean(channel_attention, axis=-1, keepdims=True)
        max_pool = tf.reduce_max(channel_attention, axis=-1, keepdims=True)
        concat = tf.concat([avg_pool, max_pool], axis=-1)

        spatial_attention = self.spatial_conv(concat)
        spatial_attention = Multiply()([channel_attention, spatial_attention])

        return spatial_attention

# Apply CBAM Block
texture_attn = CBAMBlock()(texture_base)

# ✅ Global Pooling + Dense Layer
texture_x = GlobalAveragePooling2D()(texture_attn)
texture_x = Dense(256, activation='relu')(texture_x)




Branch 2: Color-Focused Branch (ViT-based Transformer backbone)
Instead of convolutional layers, use a pretrained Vision Transformer (ViT Small or Swin Transformer) to capture rich color and global context features effectively:

In [120]:
from transformers import TFAutoModel

# Load Pretrained ViT Model
vit_model = TFAutoModel.from_pretrained("google/vit-base-patch16-224-in21k")

# ✅ Fix: Convert Input to Channel-First Format for ViT
def vit_feature_extraction(image):
    image = tf.transpose(image, [0, 3, 1, 2])  # Convert (B, H, W, C) → (B, C, H, W)
    outputs = vit_model(image, training=False)
    return outputs.last_hidden_state  # Extract features

# ✅ Wrap Inside Lambda Layer with Correct Output Shape
color_features = tf.keras.layers.Lambda(
    lambda x: vit_feature_extraction(x),
    output_shape=(197, 768)  # Ensure correct output shape
)(color_input)

# ✅ Pooling + Dense Layer
color_x = GlobalAveragePooling1D()(color_features)
color_x = Dense(256, activation='relu')(color_x)


All PyTorch model weights were used when initializing TFViTModel.

All the weights of TFViTModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.


 Branch 3: Structure Branch (Swin Transformer) (Highly Novel & High-performing)
Swin Transformer is highly effective for global context and capturing complex image-level interactions, perfect for your use case:

Implementation (using pretrained Swin Transformer from Hugging Face):

In [137]:
from transformers import TFAutoModel
import tensorflow as tf
from tensorflow.keras.layers import Lambda

# Load Pretrained Swin Transformer
swin_model = TFAutoModel.from_pretrained("microsoft/swin-tiny-patch4-window7-224")

# Define Input for Structure Branch
structure_input = tf.keras.Input(shape=(224, 224, 3))

# ✅ Force Swin Transformer to Run in Eager Execution Mode
@tf.function(experimental_relax_shapes=True)
def swin_feature_extraction(image):
    image = tf.transpose(image, [0, 3, 1, 2])  # Convert to (B, C, H, W)
    outputs = swin_model(image, training=False)
    return outputs.last_hidden_state  # Extract features

# ✅ Ensure Swin Transformer runs in Pure Eager Mode using `tf.py_function()`
structure_features = Lambda(
    lambda x: tf.py_function(swin_feature_extraction, [x], Tout=tf.float32),
    output_shape=(49, 768)  # Manually define the expected shape
)(structure_input)

# ✅ Pooling + Dense Layer
structure_x = tf.keras.layers.GlobalAveragePooling1D()(structure_features)
structure_x = tf.keras.layers.Dense(256, activation='relu')(structure_x)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFSwinModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing TFSwinModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFSwinModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFSwinModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFSwinModel for predictions without further training.


In [138]:
for images, labels in train_multi_input.take(1):
    print(f"Texture Input Shape: {images[0].shape}")   # (64, 224, 224, 3)
    print(f"Color Input Shape: {images[1].shape}")     # (64, 224, 224, 3)
    print(f"Structure Input Shape (Swin): {images[2].shape}") # (64, 224, 224, 3)


Texture Input Shape: (64, 224, 224, 3)
Color Input Shape: (64, 224, 224, 3)
Structure Input Shape (Swin): (64, 224, 224, 3)


In [169]:
print("Shape of attention_features BEFORE Reshape:", attention_features.shape)


Shape of attention_features BEFORE Reshape: (None, 1, 768)


In [168]:
from tensorflow.keras.layers import Concatenate, MultiHeadAttention, Flatten, Dropout

from tensorflow.keras.layers import Lambda

# ✅ Concatenate Features from All Three Branches
combined_features = Concatenate()([texture_x, color_x, structure_x])

expanded_features = Lambda(lambda x: tf.expand_dims(x, axis=1))(combined_features)

# ✅ Apply Multi-Head Attention
attention_features = MultiHeadAttention(num_heads=4, key_dim=64)(
    query=expanded_features,
    value=expanded_features,
    key=expanded_features
)

from tensorflow.keras.layers import Lambda


attention_features = Lambda(lambda x: tf.expand_dims(x, axis=1))(attention_features)

print("Shape of attention_features BEFORE Pooling:", attention_features.shape)

# Fix shape issues before pooling
if len(attention_features.shape) == 2:  # (batch_size, feature_dim)
    attention_features = Lambda(lambda x: tf.expand_dims(x, axis=1))(attention_features)
elif len(attention_features.shape) == 4:  # (batch_size, 1, 1, feature_dim)
    attention_features = Lambda(lambda x: tf.squeeze(x, axis=1))(attention_features)

# Apply Global Average Pooling
fusion_output = GlobalAveragePooling1D()(attention_features)
print("Shape of fusion_output AFTER Pooling:", fusion_output.shape)




# Fully Connected Layers
x = Flatten()(fusion_output)  # ✅ Ensure it flattens correctly
x = Dropout(0.4)(x)
x = Dense(256, activation='relu')(x)  # ✅ Will no longer have shape issues
x = Dropout(0.2)(x)
output = Dense(30, activation='softmax')(x)  # Final classification layer (30 classes)
  # Final classification layer (30 classes)

from tensorflow.keras.models import Model

# ✅ Create Multi-Input Model
model = Model(inputs=[texture_input, color_input, structure_input], outputs=output)



Shape of attention_features BEFORE Pooling: (None, 1, 1, 768)
Shape of fusion_output AFTER Pooling: (None, 768)


In [170]:
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import Precision, Recall

# Define loss and optimizer
loss_fn = CategoricalCrossentropy()
optimizer = AdamW(learning_rate=1e-4, weight_decay=1e-4)

# Compile the model
model.compile(
    optimizer=optimizer,
    loss=loss_fn,
    metrics=['accuracy', Precision(), Recall()]
)


In [171]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Stop training when validation loss stops improving
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Reduce learning rate if validation loss plateaus
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# Save the best model based on validation loss
checkpoint = ModelCheckpoint("best_model.h5", monitor='val_loss', save_best_only=True)

# Callback List
callbacks = [early_stopping, reduce_lr, checkpoint]


In [176]:
history = model.fit(
    train_multi_input,
    epochs=50,
    validation_data=val_multi_input,
    callbacks=[early_stopping, reduce_lr, checkpoint],
    shuffle=True)

# Plotting training and validation loss and accuracy
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


Epoch 1/50


ValueError: Exception encountered when calling Functional.call().

[1mas_list() is not defined on an unknown TensorShape.[0m

Arguments received by Functional.call():
  • inputs=('tf.Tensor(shape=(None, 224, 224, 3), dtype=float32)', 'tf.Tensor(shape=(None, 224, 224, 3), dtype=float32)', 'tf.Tensor(shape=(None, 224, 224, 3), dtype=float32)')
  • training=True
  • mask=('None', 'None', 'None')

In [ ]:
loss, accuracy, precision, recall, f1_score = model.evaluate(val_generator)
print(f'test Loss: {loss}, test Accuracy: {accuracy}')

In [160]:
# Define three input branches
model = tf.keras.Model(
    inputs=[texture_input, color_input, structure_input],  # ✅ Three separate inputs
    outputs=output
)

# ✅ Print Model Summary
model.summary()


Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ texture_input             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ efficientnetb4            │ (None, 7, 7, 1792)     │     17,673,823 │ texture_input[0][0]    │
│ (Functional)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ color_input (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_19            │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cbam_block_1 (CBAMBlock)  │ (None, 7, 7, 1792)     │      1,605,730 │ efficientnetb4[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda_14 (Lambda)        │ (None, 197, 768)       │              0 │ color_input[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda_24 (Lambda)        │ (None, 49, 768)        │              0 │ input_layer_19[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 1792)           │              0 │ cbam_block_1[0][0]     │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d… │ (None, 768)            │              0 │ lambda_14[0][0]        │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d… │ (None, 768)            │              0 │ lambda_24[0][0]        │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_38 (Dense)          │ (None, 256)            │        459,008 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_39 (Dense)          │ (None, 256)            │        196,864 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_44 (Dense)          │ (None, 256)            │        196,864 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_17            │ (None, 768)            │              0 │ dense_38[0][0],        │
│ (Concatenate)             │                        │                │ dense_39[0][0],        │
│                           │                        │                │ dense_44[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda_35 (Lambda)        │ (None, 1, 768)         │              0 │ concatenate_17[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_

 Total params: 21,124,831 (80.58 MB)

 Trainable params: 20,999,624 (80.11 MB)

 Non-trainable params: 125,207 (489.09 KB)

In [175]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
def prepare_multi_input(image, label):
    return (image, image, image), label
train_multi_input = train_dataset.map(prepare_multi_input).batch(64).prefetch(tf.data.AUTOTUNE)


val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

val_multi_input = val_dataset.map(prepare_multi_input).batch(64).prefetch(tf.data.AUTOTUNE)


In [174]:
x_train.shape

(8039, 224, 224, 3)

In [177]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Dense, Dropout, Flatten, Concatenate, GlobalAveragePooling2D,
    GlobalAveragePooling1D, Lambda, MultiHeadAttention
)
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB4
from transformers import TFAutoModel

# =======================
# Texture Branch
# =======================
texture_input = Input(shape=(224, 224, 3), name="texture_input")
# Pretrained EfficientNetB4 feature extractor (without top)
texture_base = EfficientNetB4(include_top=False, weights="imagenet")(texture_input)

# Define a CBAM block as a custom layer
class CBAMBlock(tf.keras.layers.Layer):
    def __init__(self, ratio=8, **kwargs):
        super(CBAMBlock, self).__init__(**kwargs)
        self.ratio = ratio

    def build(self, input_shape):
        channel = input_shape[-1]
        # Channel Attention layers
        self.avg_dense1 = Dense(channel // self.ratio, activation='relu', use_bias=False)
        self.avg_dense2 = Dense(channel, activation='sigmoid', use_bias=False)
        self.max_dense1 = Dense(channel // self.ratio, activation='relu', use_bias=False)
        self.max_dense2 = Dense(channel, activation='sigmoid', use_bias=False)
        # Spatial Attention layer
        self.spatial_conv = tf.keras.layers.Conv2D(1, kernel_size=7, padding="same",
                                                   activation="sigmoid", use_bias=False)

    def call(self, inputs):
        # Channel Attention
        avg_pool = GlobalAveragePooling2D()(inputs)
        max_pool = tf.keras.layers.GlobalMaxPooling2D()(inputs)
        avg_dense = self.avg_dense2(self.avg_dense1(avg_pool))
        max_dense = self.max_dense2(self.max_dense1(max_pool))
        channel_attention = tf.keras.layers.Add()([avg_dense, max_dense])
        channel_attention = tf.keras.layers.Activation("sigmoid")(channel_attention)
        channel_attention = tf.keras.layers.Multiply()([inputs, channel_attention])
        # Spatial Attention
        avg_pool_spatial = tf.reduce_mean(channel_attention, axis=-1, keepdims=True)
        max_pool_spatial = tf.reduce_max(channel_attention, axis=-1, keepdims=True)
        concat = tf.concat([avg_pool_spatial, max_pool_spatial], axis=-1)
        spatial_attention = self.spatial_conv(concat)
        spatial_attention = tf.keras.layers.Multiply()([channel_attention, spatial_attention])
        return spatial_attention

# Apply CBAM to the EfficientNetB4 features
texture_attn = CBAMBlock()(texture_base)
texture_x = GlobalAveragePooling2D()(texture_attn)
texture_x = Dense(256, activation="relu")(texture_x)  # Final texture feature vector

# =======================
# Color Branch (ViT)
# =======================
color_input = Input(shape=(224, 224, 3), name="color_input")
# Load pre-trained ViT model from Hugging Face
vit_model = TFAutoModel.from_pretrained("google/vit-base-patch16-224-in21k")

def vit_feature_extraction(image):
    # Convert input from (B, H, W, C) to (B, C, H, W)
    image = tf.transpose(image, [0, 3, 1, 2])
    outputs = vit_model(image, training=False)
    return outputs.last_hidden_state  # Expected shape: (batch_size, 197, 768)

# Wrap in a Lambda using tf.py_function (note: output shape not statically inferred)
color_features = Lambda(
    lambda x: tf.py_function(vit_feature_extraction, [x], tf.float32),
    name="vit_features"  # Optionally, specify output_shape=(197,768) if known
)(color_input)
color_x = GlobalAveragePooling1D()(color_features)
color_x = Dense(256, activation="relu")(color_x)  # Final color feature vector

# =======================
# Structure Branch (Swin Transformer)
# =======================
structure_input = Input(shape=(224, 224, 3), name="structure_input")
# Load pre-trained Swin Transformer from Hugging Face
swin_model = TFAutoModel.from_pretrained("microsoft/swin-tiny-patch4-window7-224")

@tf.function(experimental_relax_shapes=True)
def swin_feature_extraction(image):
    # Convert input from (B, H, W, C) to (B, C, H, W)
    image = tf.transpose(image, [0, 3, 1, 2])
    outputs = swin_model(image, training=False)
    return outputs.last_hidden_state  # Expected shape: (batch_size, 49, 768)

# Wrap in a Lambda using tf.py_function
structure_features = Lambda(
    lambda x: tf.py_function(swin_feature_extraction, [x], tf.float32),
    name="swin_features"  # Optionally, specify output_shape=(49,768)
)(structure_input)
structure_x = GlobalAveragePooling1D()(structure_features)
structure_x = Dense(256, activation="relu")(structure_x)  # Final structure feature vector

# =======================
# Fusion Block
# =======================
# Concatenate the three 256-d feature vectors → shape (batch_size, 768)
combined_features = Concatenate(name="combined_features")([texture_x, color_x, structure_x])

# For fusion with MultiHeadAttention, we need a sequence dimension.
# Expand dims: (batch_size, 768) → (batch_size, 1, 768)
expanded_features = Lambda(lambda x: tf.expand_dims(x, axis=1), name="expand_dims")(combined_features)

# Apply Multi-Head Attention.
# Here we use key_dim=64 (so total output dimension is 64*num_heads, e.g., 256 if num_heads=4)
attention_features = MultiHeadAttention(num_heads=4, key_dim=64, name="multi_head_attention")(
    query=expanded_features,
    value=expanded_features,
    key=expanded_features
)
# Squeeze back the sequence dimension: (batch_size, 1, 768) → (batch_size, 768)
squeezed_features = Lambda(lambda x: tf.squeeze(x, axis=1), name="squeeze_dims")(attention_features)

# Use the fused vector directly as the fusion output
fusion_output = squeezed_features

# =======================
# Classification Head
# =======================
x = Flatten(name="flatten")(fusion_output)  # (batch_size, 768)
x = Dropout(0.4, name="dropout_1")(x)
x = Dense(256, activation="relu", name="dense_1")(x)
x = Dropout(0.2, name="dropout_2")(x)
output = Dense(30, activation="softmax", name="predictions")(x)  # 30 classes

# =======================
# Build & Compile Model
# =======================
model = Model(inputs=[texture_input, color_input, structure_input], outputs=output)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss="categorical_crossentropy",
              metrics=["accuracy"])
model.summary()


All PyTorch model weights were used when initializing TFViTModel.

All the weights of TFViTModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.


NotImplementedError: Exception encountered when calling Lambda.call().

[1mWe could not automatically infer the shape of the Lambda's output. Please specify the `output_shape` argument for this Lambda layer.[0m

Arguments received by Lambda.call():
  • args=('<KerasTensor shape=(None, 224, 224, 3), dtype=float32, sparse=False, name=color_input>',)
  • kwargs={'mask': 'None'}